In [2]:
import torch
import torch.nn as nn

class SentimentCNN(nn.Module):
    def __init__(self):
        super(SentimentCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # Max pooling
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # Linear layers
        self.fc1 = nn.Linear(64 * 4, 128)  # assuming the input size to the first linear layer is 64*4
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)  # output layer

        # ReLU activation
        self.relu = nn.ReLU()

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Apply convolutional layers with ReLU activation and max pooling
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))

        # Flatten the output for the linear layers
        x = x.view(x.size(0), -1)

        # Apply linear layers with ReLU activation and dropout
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.fc4(x)  # output layer

        return x

# Create the model instance
model = SentimentCNN()
model


SentimentCNN(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

In [5]:
import torch
import torch.nn as nn

class StockPricePredictionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, lstm_hidden_dim, lstm_num_layers, num_classes):
        super(StockPricePredictionModel, self).__init__()

        # Embedding layer using GloVe vectors
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.requires_grad = False  # we do not train the GloVe embeddings

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1)

        # Max pooling
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

        # LSTM layer
        self.lstm = nn.LSTM(input_size=512, hidden_size=lstm_hidden_dim, num_layers=lstm_num_layers, batch_first=True)

        # Linear layers for news feature extraction
        self.fc_news1 = nn.Linear(lstm_hidden_dim, 256)
        self.fc_news2 = nn.Linear(256, 128)

        # Linear layers for concatenated news and price features
        self.fc_combined1 = nn.Linear(128 + 1, 64)  # +1 for the scaled price
        self.fc_combined2 = nn.Linear(64, num_classes)  # output layer

        # ReLU activation
        self.relu = nn.ReLU()

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, news, scaled_price):
        # Embedding layer
        news = self.embedding(news)

        # Convolutional layers with ReLU activation and max pooling
        news = news.permute(0, 2, 1)  # batch_size x text_length x embedding_dim -> batch_size x embedding_dim x text_length
        news = self.pool(self.relu(self.conv1(news)))
        news = self.pool(self.relu(self.conv2(news)))
        news = self.pool(self.relu(self.conv3(news)))

        # LSTM layer
        news = news.permute(0, 2, 1)  # batch_size x embedding_dim x reduced_text_length -> batch_size x reduced_text_length x embedding_dim
        news, (hn, cn) = self.lstm(news)
        news_features = news[:, -1, :]  # taking the last time step's output

        # Pass through linear layers for news
        news_features = self.dropout(self.relu(self.fc_news1(news_features)))
        news_features = self.dropout(self.relu(self.fc_news2(news_features)))

        # Concatenate the scaled price with the news features
        combined_features = torch.cat((news_features, scaled_price.unsqueeze(1)), dim=1)

        # Pass through linear layers for combined features
        combined_features = self.dropout(self.relu(self.fc_combined1(combined_features)))
        predictions = self.fc_combined2(combined_features)  # no activation, as this might be a regression problem

        return predictions

# Parameters for the model (to be adjusted as necessary)
vocab_size = 10000  # size of the vocabulary
embedding_dim = 300  # dimensionality of GloVe vectors
lstm_hidden_dim = 64  # LSTM hidden dimensions
lstm_num_layers = 1  # number of layers in LSTM
num_classes = 1  # for price prediction, we output a single value

# Create the model instance
model = StockPricePredictionModel(vocab_size, embedding_dim, lstm_hidden_dim, lstm_num_layers, num_classes)
model

# Note: The actual GloVe embeddings need to be loaded into the model's embedding layer.
# This code assumes that the 'embedding' attribute will be populated with pre-trained GloVe vectors outside of this definition.


StockPricePredictionModel(
  (embedding): Embedding(10000, 300)
  (conv1): Conv1d(300, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (lstm): LSTM(512, 64, batch_first=True)
  (fc_news1): Linear(in_features=64, out_features=256, bias=True)
  (fc_news2): Linear(in_features=256, out_features=128, bias=True)
  (fc_combined1): Linear(in_features=129, out_features=64, bias=True)
  (fc_combined2): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

In [7]:
import torch
import torch.nn as nn

class DJIAPricePredictionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(DJIAPricePredictionModel, self).__init__()
        
        # GRU layer
        self.gru = nn.GRU(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        
        # Linear layer that maps from hidden state space to price prediction space
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, historical_prices, news_sentiments):
        # Concatenate historical prices and news sentiments along feature dimension
        combined_input = torch.cat((historical_prices.unsqueeze(-1), news_sentiments.unsqueeze(-1)), dim=2)

        # Passing in the input and hidden state into the model and obtaining outputs
        gru_out, _ = self.gru(combined_input)
        
        # Taking the last time step's output for the final price prediction
        last_step_output = gru_out[:, -1, :]
        
        # Passing the last time step's output to the linear layer
        price_prediction = self.fc(last_step_output)
        return price_prediction

# Parameters for the model
input_dim = 2  # historical price and sentiment score
hidden_dim = 64  # number of features in the hidden state of the GRU
num_layers = 1  # number of stacked GRU layers
output_dim = 1  # single predicted price value

# Create the model instance
djia_price_model = DJIAPricePredictionModel(input_dim, hidden_dim, num_layers, output_dim)
djia_price_model

# Note: This model assumes that the historical prices and news sentiment scores are preprocessed and provided as inputs to the model. The sentiment scores should be derived using a lexicon-based method like SentiWordNet before being passed to the model.


DJIAPricePredictionModel(
  (gru): GRU(2, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [9]:
class FinBERTPricePredictionModel(nn.Module):
    def __init__(self, hidden_dim, num_layers, output_dim):
        super(FinBERTPricePredictionModel, self).__init__()
        
        # Since FinBERT already provides a sentiment score, there is no need for an embedding layer here.
        
        # GRU layer
        self.gru = nn.GRU(input_size=2, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        
        # Linear layer that maps from hidden state space to price prediction space
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, last_price, sentiment_score):
        # Concatenate last price and sentiment score along feature dimension
        combined_input = torch.cat((last_price.unsqueeze(-1), sentiment_score.unsqueeze(-1)), dim=2)

        # Passing in the input to the GRU and obtaining outputs
        gru_out, _ = self.gru(combined_input)
        
        # Taking the last time step's output for the final price prediction
        last_step_output = gru_out[:, -1, :]
        
        # Passing the last time step's output to the linear layer
        price_prediction = self.fc(last_step_output)
        return price_prediction

# Parameters for the model
hidden_dim = 64  # number of features in the hidden state of the GRU
num_layers = 1  # number of stacked GRU layers
output_dim = 1  # single predicted price value

# Create the model instance
finbert_price_model = FinBERTPricePredictionModel(hidden_dim, num_layers, output_dim)
finbert_price_model

# Note: This model assumes that the last price and sentiment score from FinBERT are preprocessed and provided as inputs to the model. The sentiment score should be encoded as a number between 0 and 1, where 0 could represent negative sentiment and 1 could represent positive sentiment. Neutral sentiment can be represented by a value in the middle, such as 0.5.


FinBERTPricePredictionModel(
  (gru): GRU(2, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [10]:
class TimeSeriesGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(TimeSeriesGRU, self).__init__()
        
        # GRU layer for time series prediction based only on historical data
        self.gru_ts = nn.GRU(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        
        # Linear layer for time series prediction
        self.fc_ts = nn.Linear(hidden_dim, output_dim)

    def forward(self, historical_data):
        # GRU for time series
        gru_out_ts, _ = self.gru_ts(historical_data)
        last_step_output_ts = gru_out_ts[:, -1, :]
        predicted_price_ts = self.fc_ts(last_step_output_ts)
        
        return predicted_price_ts

class FinalPricePredictionModel(nn.Module):
    def __init__(self, hidden_dim, num_layers, output_dim):
        super(FinalPricePredictionModel, self).__init__()
        
        # GRU layer for final prediction using past price, sentiment score, and predicted price
        self.gru_final = nn.GRU(input_size=3, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        
        # Linear layer for final prediction
        self.fc_final = nn.Linear(hidden_dim, output_dim)

        # Time series GRU model for initial price prediction based on historical data
        self.ts_model = TimeSeriesGRU(input_dim=1, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim)

    def forward(self, past_price, sentiment_score, historical_data):
        # Get the time series predicted price
        predicted_price_ts = self.ts_model(historical_data)

        # Concatenate past price, sentiment score, and time series predicted price along feature dimension
        combined_input = torch.cat((past_price.unsqueeze(-1), sentiment_score.unsqueeze(-1), predicted_price_ts.unsqueeze(-1)), dim=2)

        # GRU for final prediction
        gru_out_final, _ = self.gru_final(combined_input)
        last_step_output_final = gru_out_final[:, -1, :]
        final_predicted_price = self.fc_final(last_step_output_final)
        
        return final_predicted_price

# Parameters for the model
hidden_dim = 64  # number of features in the hidden state of the GRU
num_layers = 1  # number of stacked GRU layers
output_dim = 1  # single predicted price value

# Create the model instance
final_price_model = FinalPricePredictionModel(hidden_dim, num_layers, output_dim)
final_price_model

# Note: This model assumes that the past price, sentiment score, and historical data are preprocessed and provided as inputs to the model. The historical data should be a sequence of past prices used to predict the initial time-series based price prediction. Then, this initial prediction is combined with the past price and sentiment score for the final price prediction.


FinalPricePredictionModel(
  (gru_final): GRU(3, 64, batch_first=True)
  (fc_final): Linear(in_features=64, out_features=1, bias=True)
  (ts_model): TimeSeriesGRU(
    (gru_ts): GRU(1, 64, batch_first=True)
    (fc_ts): Linear(in_features=64, out_features=1, bias=True)
  )
)